In [1]:
import pandas as pd
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')[['age','sex','class','fare','survived']]
df.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


## 1. 피벗 테이블 함수 : pivot_table()
* 데이터 열 중에서 두 개의 열을 각각 행 인덱스와 열 인덱스로 사용하여 데이터를 조회

pivot_table 옵션 
* index : 행 인덱스
* columns : 열 인덱스
* values : 값을 지정, 기본 값: 평균
* aggfunc : 집계함수
* fill_value : NaN 값 채우기


In [7]:
pdf1 = pd.pivot_table(df,                # 피벗할 데이터프레임
                     index = 'class',    # 행 위치에 들어갈 열
                     columns = 'sex',    # 열 위치에 들어갈 열
                     values = 'age',     # 데이터로 사용할 열
                     aggfunc = 'mean')   # 데이터 집계함수
pdf1

sex,female,male
class,,
First,34.611765,41.281386
Second,28.722973,30.740707
Third,21.750000,26.507589


In [9]:
pdf2 = pd.pivot_table(df,
                     index = 'class',   
                     columns = 'sex',
                     values = 'survived',
                     aggfunc = ['mean', 'sum'])

pdf2

mean              sum     
sex       female      male female male
class                                 
First   0.968085  0.368852     91   45
Second  0.921053  0.157407     70   17
Third   0.500000  0.135447     72   47

3등석(Third)에 탄 여자(female)들의 생존여부(survived)는 평균 50%정도,<br>
1등석(First)에 탄 남자(male)들의 생존자는 모두 45명

In [11]:
pdf3 = pd.pivot_table(df,
                      index = ['class','sex'],
                      columns = 'survived',
                      values = ['age','fare'],
                      aggfunc = ['mean','max'])

pdf3

mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   
       male    27.255814  22.274211   12.204469   15.579696  74.0  45.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875  
       male     69.55   56.4958

3등석(Third)에 탄 여자(female)중 생존(survived)한 사람들의 최고(max) 연령은 63세이고,<br>
2등석(Second)에 탄 남자(male)중 사망(survived)한 사람들이 지불한 평균(mean) 요금(fare)은 19.488965이다.

## 2. 멀티 인덱스

class, age열에 대해 그룹객체를 생성한 후, 평균(mean)으로 집계

In [12]:
grouped = df.groupby(['class','sex'])
gdf = grouped.mean()
gdf

age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447

### 2-1 멀티 인덱스의 인덱싱 (indexing)
class열의 First행만 인덱싱

In [14]:
gdf.loc['First']

,age,fare,survived
sex,,,
female,34.611765,106.125798,0.968085
male,41.281386,67.226127,0.368852


처음부터 class그룹, sex그룹에 대해 바로 인덱싱 하려면, 튜플 형태로 인덱싱하면 된다.<br> **그룹객체를 생성하면 튜플형태로 저장됨**

In [15]:
gdf.loc[('First','female')]

age          34.611765
fare        106.125798
survived      0.968085
Name: (First, female), dtype: float64

### 2-2 멀티 인덱서 : .xs
sex그룹의 male값을 갖는 행을 추출, 즉 등급(class)별 male에 대한 자료를 인덱싱

In [16]:
gdf.xs('male',level='sex')

,age,fare,survived
class,,,
First,41.281386,67.226127,0.368852
Second,30.740707,19.741782,0.157407
Third,26.507589,12.661633,0.135447


판다스의 기본 데이터프레임 인덱싱 함수 loc와 iloc를 이용하려면<br>
큰 그룹부터 순차적으로 인덱싱을 해야하는데, 멀티인덱서 .xs를 이용하면 <br>
그룹 범주와 상관없이 수준(level)만 명시해주면 인덱싱이 가능하다.

### 2-3 멀티인덱스 해제
멀티 행 인덱스를 풀고 싶다면

In [18]:
gdf.reset_index()

,class,sex,age,fare,survived
0,First,female,34.611765,106.125798,0.968085
1,First,male,41.281386,67.226127,0.368852
2,Second,female,28.722973,21.970121,0.921053
3,Second,male,30.740707,19.741782,0.157407
4,Third,female,21.750000,16.118810,0.500000
5,Third,male,26.507589,12.661633,0.135447


컬럼이 멀티인덱스인 경우

In [19]:
gdf2 = df.groupby('class').agg(['mean','max'])[['age','fare']]
gdf2

age             fare          
             mean   max       mean       max
class                                       
First   38.233441  80.0  84.154687  512.3292
Second  29.877630  70.0  20.662183   73.5000
Third   25.140620  74.0  13.675550   69.5500

In [22]:
gdf2.columns = ['age_mean','age_max','fare_mean','fare_max']
gdf2.head()

,age_mean,age_max,fare_mean,fare_max
class,,,,
First,38.233441,80.0,84.154687,512.3292
Second,29.877630,70.0,20.662183,73.5000
Third,25.140620,74.0,13.675550,69.5500


**컬럼명을 새로 지정**해주면 멀티인덱스가 해제된다. 

### 2-4 피벗테이블의 멀티인덱싱 추가 응용

In [23]:
pdf3 = pd.pivot_table(df,
                     index = ['class','sex'],
                     columns = 'survived',
                     values = ['age','fare'],
                     aggfunc = ['mean','max'])

pdf3

mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   
       male    27.255814  22.274211   12.204469   15.579696  74.0  45.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875  
       male     69.55   56.4958

대범주 class 열의 First 그룹 인덱싱

In [24]:
pdf3.xs('First')

mean                                      max                \
                age                   fare               age          fare   
survived          0          1           0           1     0     1       0   
sex                                                                          
female    25.666667  34.939024  110.604167  105.978159  50.0  63.0  151.55   
male      44.581967  36.248000   62.894910   74.637320  71.0  80.0  263.00   

                    
                    
survived         1  
sex                 
female    512.3292  
male      512.3292

두번째 행 인덱스 범주 male 을 멀티인덱서 `xs`로 인덱싱 (`level='sex'` 옵션 필수)

In [27]:
pdf3.xs('male', level='sex')

mean                                    max                \
                age                  fare              age          fare   
survived          0          1          0          1     0     1       0   
class                                                                      
First     44.581967  36.248000  62.894910  74.637320  71.0  80.0  263.00   
Second    33.369048  16.022000  19.488965  21.095100  70.0  62.0   73.50   
Third     27.255814  22.274211  12.204469  15.579696  74.0  45.0   69.55   

                    
                    
survived         1  
class               
First     512.3292  
Second     39.0000  
Third      56.4958

In [33]:
pdf3.xs(['Second','male'], level= [0,'sex'])

mean                               max                  
                   age               fare            age        fare      
survived             0       1          0        1     0     1     0     1
class  sex                                                                
Second male  33.369048  16.022  19.488965  21.0951  70.0  62.0  73.5  39.0

여기서 레벨(level)에 사용된 0 은 칼럼명이 아니라 **행인덱스의 레벨**을 의미한다.<br>
0은 'class', 1은 'sex'를 의미하며, 'sex'대신 1을 입력해도 같은 결과이다. 

멀티인덱서 `.xs`에 level 을 추가하냐 안하냐의 차이는 범주의 크기에 따른 것도 있지만, <br> 
**반환되는 객체의 형태**에 따른 차이도 있다.   


In [28]:
pdf3.xs(('First','female'))

            survived
mean  age   0            25.666667
            1            34.939024
      fare  0           110.604167
            1           105.978159
max   age   0            50.000000
            1            63.000000
      fare  0           151.550000
            1           512.329200
Name: (First, female), dtype: float64

* level을 지정해주면 데이터프레임 객체를 반환한다

In [29]:
pdf3.xs(('First','female'), level=['class','sex'])

mean                                      max        \
                    age                   fare               age         
survived              0          1           0           1     0     1   
class sex                                                                
First female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   

                                
                fare            
survived           0         1  
class sex                       
First female  151.55  512.3292

### 열의 멀티인덱스를 인덱싱
행인덱스 인덱싱방법과 동일하며 axis=1옵션만 추가

In [31]:
# mean 열만 인덱싱
pdf3.xs('mean', axis=1)

age                   fare            
survived               0          1           0           1
class  sex                                                 
First  female  25.666667  34.939024  110.604167  105.978159
       male    44.581967  36.248000   62.894910   74.637320
Second female  36.000000  28.080882   18.250000   22.288989
       male    33.369048  16.022000   19.488965   21.095100
Third  female  23.818182  19.329787   19.773093   12.464526
       male    27.255814  22.274211   12.204469   15.579696

In [35]:
# mean열의 age열 인덱싱
pdf3.xs(('mean','age'), axis=1)

survived               0          1
class  sex                         
First  female  25.666667  34.939024
       male    44.581967  36.248000
Second female  36.000000  28.080882
       male    33.369048  16.022000
Third  female  23.818182  19.329787
       male    27.255814  22.274211